In [ ]:
#zaimportowanie wszystkich potrzebnych do analizy warstw
gmina = r"C:\Rok3\analizyprzestrzenne\1\granice_gminy.shp"
gmina_bufor = r"C:\Rok3\analizyprzestrzenne\bufor.shp"
rzeki = r"C:\Rok3\analizyprzestrzenne\swrs.shp"
lasy = r"C:\Rok3\analizyprzestrzenne\ptlz.shp"
budynki = r"C:\Rok3\analizyprzestrzenne\1\budynki_egib.gpkg"
zbwodne=r"C:\Rok3\analizyprzestrzenne\ptwp.shp"
gazociag = r"C:\Rok3\analizyprzestrzenne\gazociag.shp"
drogi = r"C:\Rok3\analizyprzestrzenne\skjz_przyciete (1).gpkg\main.skjz_przyciete"
dzialki = r"C:\Rok3\analizyprzestrzenne\dzialki.shp"
budynkimieszkalne=r"C:\Rok3\analizyprzestrzenne\1\main.budynki_egib selection 1.lyrx"

In [ ]:
#zamiportowanie bibliotek i określenie workspace
import arcpy
from arcpy import env
from arcpy.sa import *
env.workspace = "C:\Rok3\analizyprzestrzenne\1"

In [ ]:
#wykonanie map odległości dla poszczególnych warstw bazujących na odległościach i zapis do formatu geotiff

In [ ]:
EucRzeki = arcpy.sa.EucDistance(rzeki)
EucRzeki.save("C:\Rok3\analizyprzestrzenne\1\EucRzeki.tif")

In [ ]:
EucZbwodne = arcpy.sa.EucDistance(zbwodne)
EucZbwodne.save("C:\Rok3\analizyprzestrzenne\1\EucZbwodne.tif")

In [ ]:
EucGaz = arcpy.sa.EucDistance(gazociag)
EucGaz.save("C:\Rok3\analizyprzestrzenne\1\EucGaz.tif")

In [ ]:
EucDrogi = arcpy.sa.EucDistance(drogi)
EucDrogi.save("C:\Rok3\analizyprzestrzenne\1\EucDrogi.tif")

In [ ]:
EucBudynkim = arcpy.sa.EucDistance(budynkimieszkalne)
EucBudynkim.save("C:\Rok3\analizyprzestrzenne\1\EucBudynkim.tif")

In [ ]:
#wybranie z NMT tylko tych wartości, które są różne od zera
NMT = arcpy.ia.Con(
    in_conditional_raster="NMT.tif",
    in_true_raster_or_constant="NMT.tif",
    in_false_raster_or_constant=None,
    where_clause="VALUE <> 0"
)

In [ ]:
#tworzenie mapy spadków dla NMT
spadki = arcpy.sa.Slope(
    in_raster="NMT",
    output_measurement="PERCENT_RISE",
    z_factor=1,
    method="PLANAR",
    z_unit="METER",
    analysis_target_device="GPU_THEN_CPU"
)

In [ ]:
#tworzenie mapy wystawy słonecznej dla NMT
Aspect = arcpy.sa.Aspect(
    in_raster="NMT",
    method="PLANAR",
    z_unit="METER",
    project_geodesic_azimuths="GEODESIC_AZIMUTHS",
    analysis_target_device="GPU_THEN_CPU"
)

In [ ]:
#reklasyfikacja wartości - logika rozmyta dla stworzonych map odległości, w funkcji FuzzyMembership
#każda warstwa jest przycinana do obszaru gminy, natomiast wartości w FuzzyLinear zostały 
#określone na podstawie kryteriów tak, by otrzymać optymanle wyniki
#po reklasyfikacji zapis wszystkich warstw do formatu geotiff

In [ ]:
fuz_bud1 = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(EucBudynkim, gmina), FuzzyLinear(10,25))
fuz_bud2 = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(EucBudynkim, gmina), FuzzyLinear(200,150))
fuz_budynki = arcpy.sa.FuzzyOverlay([fuz_bud1,fuz_bud2],"AND")
fuz_budynki.save(r"C:\Rok3\analizyprzestrzenne\1\fuz_budynki.tif")

In [ ]:
fuz_drogi1 = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(EucDrogi, gmina), FuzzyLinear(5,15))
fuz_drogi2 = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(EucDrogi, gmina), FuzzyLinear(200,100))
fuz_drogi = arcpy.sa.FuzzyOverlay([fuz_drogi1,fuz_drogi2],"AND")
fuz_drogi.save(r"C:\Rok3\analizyprzestrzenne\1\fuz_drogi.tif")

In [ ]:
fuz_rzeki1 = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(EucRzeki, gmina), FuzzyLinear(20,25))
fuz_rzeki2 = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(EucRzeki, gmina), FuzzyLinear(200,25))
fuz_wod1 = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(EucZbwodne, gmina), FuzzyLinear(20,25))
fuz_wod2 = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(EucZbwodne, gmina), FuzzyLinear(200,25))
fuz_wodypow = arcpy.sa.FuzzyOverlay([fuz_wod1,fuz_wod2],"AND")
fuz_rzeki = arcpy.sa.FuzzyOverlay([fuz_rzeki1,fuz_rzeki2],"AND")
fuz_wody = arcpy.sa.FuzzyOverlay([fuz_wodypow,fuz_rzeki],"OR")
fuz_wody.save(r"C:\Rok3\analizyprzestrzenne\1\fuz_wody.tif")

In [ ]:
fuz_gazociag = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(EucGaz, gmina),FuzzyLinear(25,500))
fuz_gazociag.save(r"C:\Rok3\analizyprzestrzenne\1\fuz_gazociag.tif")

In [ ]:
fuz_spadki = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(spadki, gmina),FuzzyLinear(20,0))
fuz_spadki.save(r"C:\Rok3\analizyprzestrzenne\1\fuz_spadki.tif")

In [ ]:
stoki = arcpy.sa.ExtractByMask(Aspect,gmina)
fuz_stoki1 = arcpy.sa.FuzzyMembership(stoki,FuzzyLinear(112.5,135))
fuz_stoki2 = arcpy.sa.FuzzyMembership(stoki,FuzzyLinear(247.5,225))
fuz_stoki = arcpy.sa.FuzzyOverlay([fuz_stoki1,fuz_stoki2],"AND")
fuz_stoki.save(r"C:\Rok3\analizyprzestrzenne\1\fuz_stoki.tif")

In [ ]:
#zamiana warstwy z lasami z rastru na poligon
lasyRaster = arcpy.conversion.PolygonToRaster(
    in_features=lasy,
    value_field="RODZAJ",
    out_rasterdataset=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\lasyRaster",
    cell_assignment="CELL_CENTER",
    priority_field="NONE",
    cellsize=25,
    build_rat="BUILD"
)

In [ ]:
#przycięcie powstałej warstwy poligonowej do granic gminy
las1 = arcpy.sa.ExtractByMask(lasyRaster, gmina)

In [ ]:
#reklasyfikacja wartości - podejście ostre
Reclass_lasy = arcpy.sa.Reclassify(
    in_raster="las1",
    reclass_field="RODZAJ",
    remap="las 0;zagajnik 0;Las 0;Zdr 0;Zag 0;NODATA 1",
    missing_values="DATA"
)
Reclass_lasy = arcpy.sa.ExtractByMask(Reclass_lasy, gmina)
Reclass_lasy.save(r"C:\Rok3\analizyprzestrzenne\1\Reclass_lasy.tif")

In [ ]:
#stworzenmie mapy intensywności dla warstwy z hotelami
hotelrating = arcpy.sa.KernelDensity(
    in_features="hotele",
    population_field="rating",
    cell_size=10,
    search_radius=None,
    area_unit_scale_factor="SQUARE_KILOMETERS",
    out_cell_values="DENSITIES",
    method="PLANAR",
    in_barriers=None
)

In [ ]:
#wartość srednia rastra z powstałej mapy intensywności dla hoteli
mean_value = arcpy.management.GetRasterProperties(hotelrating, "MEAN")[0]

In [ ]:
#reklasyfikacja wartości - logika rozmyta dla mapy intensywności hoteli, w funkcji FuzzyMembership
#warstwa jest przycinana do obszaru gminy, natomiast wartości w narzędziu zostały 
#określone na podstawie kryteriów tak, by otrzymać optymanle wyniki
#po reklasyfikacji zapis warstwy do formatu geotiff
fuz_hotele = arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(hotelrating, gmina), f'LINEAR {mean_value} 0', 'NONE')
fuz_hotele.save(r"C:\Rok3\analizyprzestrzenne\1\fuz_hotele.tif")

In [ ]:
#wykonanie Service Area z czasem przejazdu w obszarach wiejskich z przedziałami wartości [2,4,6,8,10,12,14]
arcpy.na.MakeServiceAreaAnalysisLayer(
    network_data_source="https://www.arcgis.com/",
    layer_name="Service Area 2",
    travel_mode="Czas przejazdu w obszarach wiejskich",
    travel_direction="FROM_FACILITIES",
    cutoffs=[2,4,6,8,10,12,14],
    time_of_day=None,
    time_zone="LOCAL_TIME_AT_LOCATIONS",
    output_type="POLYGONS_AND_LINES",
    polygon_detail="STANDARD",
    geometry_at_overlaps="OVERLAP",
    geometry_at_cutoffs="RINGS",
    polygon_trim_distance="100 Meters",
    exclude_sources_from_polygon_generation=None,
    accumulate_attributes=None,
    ignore_invalid_locations="SKIP"
)

In [ ]:
#dodanie restauracji do Service Area
arcpy.na.AddLocations(
    in_network_analysis_layer="Service Area 2",
    sub_layer="Facilities",
    in_table="rest",
    field_mappings="Name Name #;CurbApproach # 0;Attr_Minutes # 0;Attr_TravelTime # 0;Attr_Miles # 0;Attr_Kilometers # 0;Attr_TimeAt1KPH # 0;Attr_WalkTime # 0;Attr_TruckMinutes # 0;Attr_TruckTravelTime # 0;Breaks_Minutes # #;Breaks_TravelTime # #;Breaks_Miles # #;Breaks_Kilometers # #;Breaks_TimeAt1KPH # #;Breaks_WalkTime # #;Breaks_TruckMinutes # #;Breaks_TruckTravelTime # #",
    search_tolerance="20000 Meters",
    sort_field=None,
    search_criteria="main.Routing_Streets SHAPE",
    match_type="MATCH_TO_CLOSEST",
    append="APPEND",
    snap_to_position_along_network="NO_SNAP",
    snap_offset="2 Meters",
    exclude_restricted_elements="EXCLUDE",
    search_query=None,
    allow_auto_relocate="ALLOW"
)

In [ ]:
#zapis warstwy z Service Area dla restauracji
arcpy.na.Solve(
    in_network_analysis_layer="Service Area 2",
    ignore_invalids="SKIP",
    terminate_on_solve_error="TERMINATE",
    simplification_tolerance=None,
    overrides=""
)

In [ ]:
#zamiana powstałej warstwy Service Area z restauracjami z poligonu na raster
Polygons_PolygonToRasterrest= arcpy.conversion.PolygonToRaster(
    in_features=r'Service Area 2\\Polygons',
    value_field="ToBreak",
    out_rasterdataset=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\Polygons_PolygonToRasterrest",
    cell_assignment="CELL_CENTER",
    priority_field="NONE",
    cellsize=25,
    build_rat="BUILD")

In [ ]:
#najmniejsza wartość z rastra zawierającego restauracje
min_value1 = arcpy.management.GetRasterProperties(Polygons_PolygonToRasterrest, "MINIMUM")[0]

In [ ]:
#reklasyfikacja wartości - logika rozmyta dla warstwy rastrowej z restauracjami, w funkcji FuzzyMembership
#warstwa jest przycinana do obszaru gminy, natomiast wartości w narzędziu zostały 
#określone na podstawie kryteriów tak, by otrzymać optymanle wyniki
#po reklasyfikacji zapis warstwy do formatu geotiff
fuzzy_rest= arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(Polygons_PolygonToRasterrest, gmina), f'LINEAR 15 {min_value1}', 'NONE')
fuzzy_rest.save(r"C:\Rok3\analizyprzestrzenne\1\fuzzy_rest.tif")

In [ ]:
#zamiana powstałej warstwy Service Area z restauracjami z polyline na raster
Lines_PolylineToRasterrest = arcpy.conversion.PolylineToRaster(
    in_features=r'Service Area 2\\Lines',
    value_field="ToCumul_TravelTime",
    out_rasterdataset=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\Lines_PolylineToRasterrest",
    cell_assignment="MAXIMUM_LENGTH",
    priority_field="NONE",
    cellsize=25,
    build_rat="BUILD"
)

In [ ]:
#najmniejsza wartość z rastra zawierającego restauracje
min_value2 = arcpy.management.GetRasterProperties(Lines_PolylineToRasterrest, "MINIMUM")[0]

In [ ]:
#reklasyfikacja wartości - logika rozmyta dla warstwy rastrowej (wcześniej polyline) z restauracjami, w funkcji FuzzyMembership
#warstwa jest przycinana do obszaru gminy, natomiast wartości w narzędziu zostały 
#określone na podstawie kryteriów tak, by otrzymać optymanle wyniki
#po reklasyfikacji zapis warstwy do formatu geotiff
fuzzy_rest_lines= arcpy.sa.FuzzyMembership(arcpy.sa.ExtractByMask(Lines_PolylineToRasterrest, gmina), f'LINEAR 15 {min_value2}', 'NONE')
fuzzy_rest_lines.save(r"C:\Rok3\analizyprzestrzenne\1\fuzzy_rest_lines.tif")

In [ ]:
#połączenie wyników dla kryteriów, dla których zastosowano logikę rozmytą - równe wagi dla wszystkich kryteriów
in_rasters = "fuz_hotele VALUE 0.125;fuzzy_rest VALUE 0.125;fuz_spadki VALUE 0.125;fuz_stoki VALUE 0.125;fuz_wody VALUE 0.125;fuz_drogi VALUE 0.125;fuz_budynki VALUE 0.125;fuz_gazociag VALUE 0.125"
rowne = arcpy.ia.WeightedSum(in_rasters)

In [ ]:
#połączenie wyniku łączenia kryteriów rozmytych z kryterium ostrym
niewagowane = arcpy.sa.FuzzyOverlay(
    in_rasters="rowne;Reclass_lasy",
    overlay_type="AND",
    gamma=0.9
)

In [ ]:
#zapis połączonego wyniku
niewagowane.save(r"C:\Rok3\analizyprzestrzenne\1\niewagowane.tif")

In [ ]:
#połączenie wyników dla kryteriów, dla których zastosowano logikę rozmytą - różne wagi dla kryteriów
in_rasters = "fuz_budynki VALUE 0.2;fuzzy_rest VALUE 0.1;fuz_hotele Value 0.15;fuz_stoki VALUE 0.05;fuz_spadki VALUE 0.1;fuz_gazociag VALUE 0.05;fuz_wody VALUE 0.2;fuz_drogi VALUE 0.15"
rozne = arcpy.ia.WeightedSum(in_rasters)

In [ ]:
#połączenie wyniku łączenia kryteriów rozmytych z kryterium ostrym
wagowane = arcpy.sa.FuzzyOverlay(
    in_rasters="rozne;Reclass_lasy",
    overlay_type="AND",
    gamma=0.9
)

In [ ]:
#zapis połączonego wyniku
wagowane.save(r"C:\Rok3\analizyprzestrzenne\1\wagowane.tif")

In [ ]:
#reklasyfikacja wyniku z jednakowymi wagami
dzialkipow70 = arcpy.ddd.Reclassify(
    in_raster="niewagowane",
    reclass_field="VALUE",
    remap="0 0,700000 0;0,700000 1 1",
    out_raster=r"C:\Rok3\analizyprzestrzenne\1\dzialkpow70",
    missing_values="DATA"
)

In [ ]:
#zamiana powstałego rastra na poligon
arcpy.conversion.RasterToPolygon(
    in_raster="dzialkpow70",
    out_polygon_features=r"C:\Rok3\analizyprzestrzenne\1\Poly_dz70",
    raster_field="VALUE",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
)

In [ ]:
#wybór tych części warstwy poligonowej gdzie gridcode jest równy 1
#zapis części do osobnej warstwy
obszar = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="Poly_dz70",
    selection_type="NEW_SELECTION",
    where_clause="gridcode = 1",
    invert_where_clause=None
)

arcpy.management.CopyFeatures(obszar,r"C:\Rok3\analizyprzestrzenne\1\obszar")

In [ ]:
#reklasyfikacja wyniku z różnymi wagami 
dzialkipow702 = arcpy.ddd.Reclassify(
    in_raster="wagowane",
    reclass_field="VALUE",
    remap="0 0,700000 0;0,700000 1 1",
    out_raster=r"C:\Rok3\analizyprzestrzenne\1\dzialkpow702",
    missing_values="DATA"
)

In [ ]:
#zamiana powstałego rastra na poligon
arcpy.conversion.RasterToPolygon(
    in_raster="dzialkpow702",
    out_polygon_features=r"C:\Rok3\analizyprzestrzenne\1\Poly_dz702",
    raster_field="VALUE",
    create_multipart_features="SINGLE_OUTER_PART",
    max_vertices_per_feature=None
)

In [ ]:
#wybór tych części warstwy poligonowej gdzie gridcode jest równy 1
#zapis części do osobnej warstwy
obszar2 = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="Poly_dz702",
    selection_type="NEW_SELECTION",
    where_clause="gridcode = 1",
    invert_where_clause=None
)

arcpy.management.CopyFeatures(obszar2,r"C:\Rok3\analizyprzestrzenne\1\obszar2.shp")

In [ ]:
#policzenie pola dla warstwy z działkami
dzialkiobszar = arcpy.management.CalculateGeometryAttributes(
    in_features=dzialki,
    geometry_property="area AREA",
    length_unit="",
    area_unit="HECTARES",
    coordinate_system='PROJCS["ETRS_1989_Poland_CS92",GEOGCS["GCS_ETRF2000_PL",DATUM["D_ETRF2000_Poland",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",-5300000.0],PARAMETER["Central_Meridian",19.0],PARAMETER["Scale_Factor",0.9993],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT"
)

In [ ]:
#wyszukanie części wspólnych obszaru działek i "obszaru" dla wyniku z równymi wagami
#zapis do osobnej warstwy shp
dzialkiniewag = arcpy.management.SelectLayerByLocation(
    in_layer=dzialki,
    overlap_type="INTERSECT",
    select_features="obszar",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)
arcpy.management.CopyFeatures(dzialkiniewag,r"C:\Rok3\analizyprzestrzenne\1\dzialkiniewag.shp")

In [ ]:
#wyszukanie części wspólnych obszaru działek i "obszaru" dla wyniku z różnymi wagami
#zapis do osobnej warstwy shp
dzialkiwag = arcpy.management.SelectLayerByLocation(
    in_layer=dzialki,
    overlap_type="INTERSECT",
    select_features="obszar2",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)
arcpy.management.CopyFeatures(dzialkiwag,r"C:\Rok3\analizyprzestrzenne\1\dzialkiwag.shp")

In [ ]:
#obliczenie jaka część działki stanowi obszar przydatny pod lokalizację inwestycji
#dla wyniku równych wag
arcpy.analysis.SummarizeWithin(
    in_polygons="dzialkiniewag",
    in_sum_features="obszar",
    out_feature_class=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\niewag_SumWith",
    keep_all_polygons="KEEP_ALL",
    sum_fields=None,
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="HECTARES",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)

In [ ]:
#obliczenie nowego pola dla warstwy SummarizeWithin z wynikiem z równymi wagami, na podstawie którego 
#będzie można wybrać działki, których min 70% obszaru stanowi obszar spełniający wszystkie kryteria
arcpy.management.CalculateField(
    in_table="niewag_SumWith",
    field="pow",
    expression="!sum_Area_HECTARES! / !area!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
#wybranie działek, których min 70% obszaru stanowi obszar spełniający wszystkie kryteria i zapis do nowej warstwy
pasujace1 = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="niewag_SumWith",
    selection_type="NEW_SELECTION",
    where_clause="pow >= 0.7",
    invert_where_clause=None
)

arcpy.management.CopyFeatures(pasujace1,r"C:\Rok3\analizyprzestrzenne\1\pasujace1.shp")

In [ ]:
#połączenie sąsiadujących działek w jeden obiekt
arcpy.management.Dissolve(
    in_features="pasujace1",
    out_feature_class=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\dzialki1_Dis",
    dissolve_field="FID",
    statistics_fields=None,
    multi_part="SINGLE_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

In [ ]:
#obliczenie jaka część działki stanowi obszar przydatny pod lokalizację inwestycji
#dla wyniku różnych wag
arcpy.analysis.SummarizeWithin(
    in_polygons="dzialkiwag",
    in_sum_features="obszar",
    out_feature_class=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\wag_SumWith",
    keep_all_polygons="KEEP_ALL",
    sum_fields=None,
    sum_shape="ADD_SHAPE_SUM",
    shape_unit="HECTARES",
    group_field=None,
    add_min_maj="NO_MIN_MAJ",
    add_group_percent="NO_PERCENT",
    out_group_table=None
)

In [ ]:
#obliczenie nowego pola dla warstwy SummarizeWithin z wynikiem z różnymi wagami, na podstawie którego 
#będzie można wybrać działki, których min 70% obszaru stanowi obszar spełniający wszystkie kryteria
arcpy.management.CalculateField(
    in_table="wag_SumWith",
    field="pow",
    expression="!sum_Area_HECTARES! / !area!",
    expression_type="PYTHON3",
    code_block="",
    field_type="DOUBLE",
    enforce_domains="NO_ENFORCE_DOMAINS"
)

In [ ]:
#wybranie działek, których min 70% obszaru stanowi obszar spełniający wszystkie kryteria i zapis do nowej warstwy
pasujace2 = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="wag_SumWith",
    selection_type="NEW_SELECTION",
    where_clause="pow >= 0.7",
    invert_where_clause=None
)
arcpy.management.CopyFeatures(pasujace2,r"C:\Rok3\analizyprzestrzenne\1\pasujace2.shp")

In [ ]:
#połączenie sąsiadujących działek w jeden obiekt
arcpy.management.Dissolve(
    in_features="pasujace2",
    out_feature_class=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\dzialki2_Dis",
    dissolve_field="FID",
    statistics_fields="pow UNIQUE",
    multi_part="SINGLE_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

In [ ]:
#wybór tych działek, których pole wynosi od 1 do 6 ha (równe wagi)
dz_wynik1 = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="dzialki1_Dis",
    selection_type="NEW_SELECTION",
    where_clause="Shape_Area >= 10000 And Shape_Area <= 60000",
    invert_where_clause=None
)
arcpy.management.CopyFeatures(dz_wynik1,r"C:\Rok3\analizyprzestrzenne\1\dz_wynik1.shp")

In [ ]:
#wybór tych działek, których pole wynosi od 1 do 6 ha (różne wagi)
dz_wynik2 = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view="dzialki2_Dis",
    selection_type="NEW_SELECTION",
    where_clause="Shape_Area >= 10000 And Shape_Area <= 60000",
    invert_where_clause=None
)
arcpy.management.CopyFeatures(dz_wynik2,r"C:\Rok3\analizyprzestrzenne\1\dz_wynik2.shp")

In [ ]:
#połączenie wszystkich warstw pokrycia terenu w jedną warstwę
arcpy.management.Merge(
    inputs="PL.PZGiK.337.0210__OT_PTGN_A;PL.PZGiK.337.0210__OT_PTKM_A;PL.PZGiK.337.0210__OT_PTLZ_A;PL.PZGiK.337.0210__OT_PTNZ_A;PL.PZGiK.337.0210__OT_PTPL_A;PL.PZGiK.337.0210__OT_PTRK_A;PL.PZGiK.337.0210__OT_PTTR_A;PL.PZGiK.337.0210__OT_PTUT_A;PL.PZGiK.337.0210__OT_PTWP_A;PL.PZGiK.337.0210__OT_PTWZ_A;PL.PZGiK.337.0210__OT_PTZB_A;PL.PZGiK.337.0210__OT_PTSO_A",
    output=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\kwzg_Merge",
    field_mappings='TERYT "TERYT" true true false 4 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTKM_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTLZ_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTNZ_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTPL_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTRK_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTTR_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTUT_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTWP_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTWZ_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTZB_A,TERYT,0,4,PL.PZGiK.337.0210__OT_PTSO_A,TERYT,0,4;LOKALNYID "LOKALNYID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTKM_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTPL_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTRK_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTTR_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTUT_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTWP_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTZB_A,LOKALNYID,0,80,PL.PZGiK.337.0210__OT_PTSO_A,LOKALNYID,0,80;WERSJAID "WERSJAID" true true false 40 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTKM_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTLZ_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTNZ_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTPL_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTRK_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTTR_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTUT_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTWP_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTWZ_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTZB_A,WERSJAID,0,40,PL.PZGiK.337.0210__OT_PTSO_A,WERSJAID,0,40;PNAZW "PNAZW" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTKM_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTPL_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTRK_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTTR_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTUT_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTWP_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTZB_A,PNAZW,0,80,PL.PZGiK.337.0210__OT_PTSO_A,PNAZW,0,80;GMLID "GMLID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTKM_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTPL_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTRK_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTTR_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTUT_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTWP_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTZB_A,GMLID,0,80,PL.PZGiK.337.0210__OT_PTSO_A,GMLID,0,80;X_KOD "X_KOD" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTKM_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTPL_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTRK_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTTR_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTUT_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTWP_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTZB_A,X_KOD,0,80,PL.PZGiK.337.0210__OT_PTSO_A,X_KOD,0,80;X_SKRKARTO "X_SKRKARTO" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTKM_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTPL_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTRK_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTTR_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTUT_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTWP_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTZB_A,X_SKRKARTO,0,80,PL.PZGiK.337.0210__OT_PTSO_A,X_SKRKARTO,0,80;X_SKRK_NIL "X_SKRK_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTKM_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTPL_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTRK_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTTR_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTUT_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTWP_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTZB_A,X_SKRK_NIL,0,80,PL.PZGiK.337.0210__OT_PTSO_A,X_SKRK_NIL,0,80;X_KATDOKG "X_KATDOKG" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTKM_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTPL_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTRK_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTTR_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTUT_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTWP_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTZB_A,X_KATDOKG,0,80,PL.PZGiK.337.0210__OT_PTSO_A,X_KATDOKG,0,80;X_DOKLGEOM "X_DOKLGEOM" true true false 2 Float 1 1,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTKM_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTLZ_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTNZ_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTPL_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTRK_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTTR_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTUT_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTWP_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTWZ_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTZB_A,X_DOKLGEOM,-1,-1,PL.PZGiK.337.0210__OT_PTSO_A,X_DOKLGEOM,-1,-1;X_ZRODLOG "X_ZRODLOG" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTKM_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTPL_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTRK_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTTR_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTUT_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTWP_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTZB_A,X_ZRODLOG,0,80,PL.PZGiK.337.0210__OT_PTSO_A,X_ZRODLOG,0,80;X_ZRODLOA "X_ZRODLOA" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTKM_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTPL_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTRK_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTTR_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTUT_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTWP_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTZB_A,X_ZRODLOA,0,80,PL.PZGiK.337.0210__OT_PTSO_A,X_ZRODLOA,0,80;X_KATIST "X_KATIST" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTKM_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTPL_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTRK_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTTR_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTUT_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTWP_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTZB_A,X_KATIST,0,80,PL.PZGiK.337.0210__OT_PTSO_A,X_KATIST,0,80;X_KATI_NIL "X_KATI_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTKM_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTPL_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTRK_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTTR_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTUT_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTWP_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTZB_A,X_KATI_NIL,0,80,PL.PZGiK.337.0210__OT_PTSO_A,X_KATI_NIL,0,80;X_RODZREPG "X_RODZREPG" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTKM_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTPL_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTRK_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTTR_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTUT_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTWP_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTZB_A,X_RODZREPG,0,80,PL.PZGiK.337.0210__OT_PTSO_A,X_RODZREPG,0,80;X_UWAGI "X_UWAGI" true true false 254 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTKM_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTLZ_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTNZ_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTPL_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTRK_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTTR_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTUT_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTWP_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTWZ_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTZB_A,X_UWAGI,0,254,PL.PZGiK.337.0210__OT_PTSO_A,X_UWAGI,0,254;X_UZYTKOWN "X_UZYTKOWN" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTKM_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTPL_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTRK_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTTR_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTUT_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTWP_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTZB_A,X_UZYTKOWN,0,80,PL.PZGiK.337.0210__OT_PTSO_A,X_UZYTKOWN,0,80;X_AKTG "X_AKTG" true true false 8 Date 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTKM_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTLZ_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTNZ_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTPL_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTRK_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTTR_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTUT_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTWP_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTWZ_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTZB_A,X_AKTG,-1,-1,PL.PZGiK.337.0210__OT_PTSO_A,X_AKTG,-1,-1;X_AKTA "X_AKTA" true true false 8 Date 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTKM_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTLZ_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTNZ_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTPL_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTRK_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTTR_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTUT_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTWP_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTWZ_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTZB_A,X_AKTA,-1,-1,PL.PZGiK.337.0210__OT_PTSO_A,X_AKTA,-1,-1;POCZWERS "POCZWERS" true true false 8 Date 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTKM_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTLZ_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTNZ_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTPL_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTRK_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTTR_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTUT_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTWP_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTWZ_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTZB_A,POCZWERS,-1,-1,PL.PZGiK.337.0210__OT_PTSO_A,POCZWERS,-1,-1;KONIECWERS "KONIECWERS" true true false 8 Date 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTKM_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTLZ_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTNZ_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTPL_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTRK_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTTR_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTUT_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTWP_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTWZ_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTZB_A,KONIECWERS,-1,-1,PL.PZGiK.337.0210__OT_PTSO_A,KONIECWERS,-1,-1;X_DATAUTW "X_DATAUTW" true true false 8 Date 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTKM_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTLZ_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTNZ_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTPL_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTRK_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTTR_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTUT_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTWP_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTWZ_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTZB_A,X_DATAUTW,-1,-1,PL.PZGiK.337.0210__OT_PTSO_A,X_DATAUTW,-1,-1;X_INFDOD "X_INFDOD" true true false 254 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTKM_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTLZ_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTNZ_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTPL_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTRK_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTTR_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTUT_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTWP_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTWZ_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTZB_A,X_INFDOD,0,254,PL.PZGiK.337.0210__OT_PTSO_A,X_INFDOD,0,254;KOD10K "KOD10K" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD10K,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD10K,0,80;KOD10K_NIL "KOD10K_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD10K_NIL,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD10K_NIL,0,80;KOD25K "KOD25K" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD25K,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD25K,0,80;KOD25K_NIL "KOD25K_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD25K_NIL,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD25K_NIL,0,80;KOD50K "KOD50K" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD50K,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD50K,0,80;KOD50K_NIL "KOD50K_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD50K_NIL,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD50K_NIL,0,80;KOD100K "KOD100K" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD100K,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD100K,0,80;KOD100_NIL "KOD100_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD100_NIL,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD100_NIL,0,80;KOD250K "KOD250K" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD250K,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD250K,0,80;KOD250_NIL "KOD250_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD250_NIL,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD250_NIL,0,80;KOD500K "KOD500K" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD500K,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD500K,0,80;KOD500_NIL "KOD500_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD500_NIL,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD500_NIL,0,80;KOD1000K "KOD1000K" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD1000K,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD1000K,0,80;KOD1000_NI "KOD1000_NI" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTKM_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTPL_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTRK_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTTR_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTUT_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTWP_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTZB_A,KOD1000_NI,0,80,PL.PZGiK.337.0210__OT_PTSO_A,KOD1000_NI,0,80;RODZAJ "RODZAJ" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,RODZAJ,0,80,PL.PZGiK.337.0210__OT_PTKM_A,RODZAJ,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,RODZAJ,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,RODZAJ,0,80,PL.PZGiK.337.0210__OT_PTRK_A,RODZAJ,0,80,PL.PZGiK.337.0210__OT_PTTR_A,RODZAJ,0,80,PL.PZGiK.337.0210__OT_PTUT_A,RODZAJ,0,80,PL.PZGiK.337.0210__OT_PTWP_A,RODZAJ,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,RODZAJ,0,80,PL.PZGiK.337.0210__OT_PTZB_A,RODZAJ,0,80,PL.PZGiK.337.0210__OT_PTSO_A,RODZAJ,0,80;PRNG_LOKID "PRNG_LOKID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTKM_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTPL_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTRK_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTTR_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTUT_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTWP_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTZB_A,PRNG_LOKID,0,80,PL.PZGiK.337.0210__OT_PTSO_A,PRNG_LOKID,0,80;PRNG_PNAZW "PRNG_PNAZW" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTKM_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTPL_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTRK_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTTR_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTUT_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTWP_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTZB_A,PRNG_PNAZW,0,80,PL.PZGiK.337.0210__OT_PTSO_A,PRNG_PNAZW,0,80;PRNG_WERID "PRNG_WERID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTKM_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTNZ_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTPL_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTRK_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTTR_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTUT_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTWP_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTWZ_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTZB_A,PRNG_WERID,0,80,PL.PZGiK.337.0210__OT_PTSO_A,PRNG_WERID,0,80;BD500_ID "BD500_ID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,BD500_ID,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,BD500_ID,0,80,PL.PZGiK.337.0210__OT_PTPL_A,BD500_ID,0,80,PL.PZGiK.337.0210__OT_PTRK_A,BD500_ID,0,80,PL.PZGiK.337.0210__OT_PTUT_A,BD500_ID,0,80,PL.PZGiK.337.0210__OT_PTWP_A,BD500_ID,0,80;BD500_PNA "BD500_PNA" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,BD500_PNA,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,BD500_PNA,0,80,PL.PZGiK.337.0210__OT_PTPL_A,BD500_PNA,0,80,PL.PZGiK.337.0210__OT_PTRK_A,BD500_PNA,0,80,PL.PZGiK.337.0210__OT_PTUT_A,BD500_PNA,0,80,PL.PZGiK.337.0210__OT_PTWP_A,BD500_PNA,0,80;BD500_WEID "BD500_WEID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTGN_A,BD500_WEID,0,80,PL.PZGiK.337.0210__OT_PTLZ_A,BD500_WEID,0,80,PL.PZGiK.337.0210__OT_PTPL_A,BD500_WEID,0,80,PL.PZGiK.337.0210__OT_PTRK_A,BD500_WEID,0,80,PL.PZGiK.337.0210__OT_PTUT_A,BD500_WEID,0,80,PL.PZGiK.337.0210__OT_PTWP_A,BD500_WEID,0,80;area "area" true true false 80 Double 2 79,First,#,PL.PZGiK.337.0210__OT_PTGN_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTKM_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTLZ_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTNZ_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTPL_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTRK_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTTR_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTUT_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTWP_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTWZ_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTZB_A,area,-1,-1,PL.PZGiK.337.0210__OT_PTSO_A,area,-1,-1;GATUNEKDRZ "GATUNEKDRZ" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTLZ_A,GATUNEKDRZ,0,80;GATDRZ_NIL "GATDRZ_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTLZ_A,GATDRZ_NIL,0,80;KATEGORIA "KATEGORIA" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTLZ_A,KATEGORIA,0,80;NAZWA "NAZWA" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTLZ_A,NAZWA,0,80;MATNAWIERZ "MATNAWIERZ" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,MATNAWIERZ,0,80;MATNAW_NIL "MATNAW_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,MATNAW_NIL,0,80;UL_EM_ID "UL_EM_ID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,UL_EM_ID,0,80;UL_EM_PN "UL_EM_PN" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,UL_EM_PN,0,80;UL_EM_WERS "UL_EM_WERS" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,UL_EM_WERS,0,80;UL_IDTERUL "UL_IDTERUL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,UL_IDTERUL,0,80;UL_IDTUL_N "UL_IDTUL_N" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,UL_IDTUL_N,0,80;UL_KODULGM "UL_KODULGM" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,UL_KODULGM,0,80;UL_NAZWA1 "UL_NAZWA1" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,UL_NAZWA1,0,80;UL_NAZWA2 "UL_NAZWA2" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,UL_NAZWA2,0,80;UL_PRZEDR1 "UL_PRZEDR1" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,UL_PRZEDR1,0,80;UL_PRZEDR2 "UL_PRZEDR2" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTPL_A,UL_PRZEDR2,0,80;EGIB_LOKID "EGIB_LOKID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTTR_A,EGIB_LOKID,0,80;EGIB_PNAZW "EGIB_PNAZW" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTTR_A,EGIB_PNAZW,0,80;EGIB_WERID "EGIB_WERID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTTR_A,EGIB_WERID,0,80;GATUNEK "GATUNEK" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTUT_A,GATUNEK,0,80;GATUN_NIL "GATUN_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTUT_A,GATUN_NIL,0,80;IDMPHP "IDMPHP" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,IDMPHP,0,80;POZIOMWODY "POZIOMWODY" true true false 5 Float 2 4,First,#,PL.PZGiK.337.0210__OT_PTWP_A,POZIOMWODY,-1,-1;KATISTNIEN "KATISTNIEN" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,KATISTNIEN,0,80;KATIST_NIL "KATIST_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,KATIST_NIL,0,80;ZB_IDPRNG "ZB_IDPRNG" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,ZB_IDPRNG,0,80;ZB_NAZWA "ZB_NAZWA" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,ZB_NAZWA,0,80;ZB_BD10_ID "ZB_BD10_ID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,ZB_BD10_ID,0,80;ZB_BD10_PN "ZB_BD10_PN" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,ZB_BD10_PN,0,80;ZB_BD10_WE "ZB_BD10_WE" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,ZB_BD10_WE,0,80;ZB_X_INFD "ZB_X_INFD" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,ZB_X_INFD,0,80;ZB_X_AKTA "ZB_X_AKTA" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,ZB_X_AKTA,0,80;ZBWODN_NIL "ZBWODN_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,ZBWODN_NIL,0,80;CI_PRNG_ID "CI_PRNG_ID" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,CI_PRNG_ID,0,80;CI_PRNG_PN "CI_PRNG_PN" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,CI_PRNG_PN,0,80;CI_PRNG_WE "CI_PRNG_WE" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,CI_PRNG_WE,0,80;CI_IDPRNG "CI_IDPRNG" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,CI_IDPRNG,0,80;CI_NAZWA "CI_NAZWA" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,CI_NAZWA,0,80;CI_DLUGOSC "CI_DLUGOSC" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,CI_DLUGOSC,0,80;CIEK3_NIL "CIEK3_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWP_A,CIEK3_NIL,0,80;SUROWIEC "SUROWIEC" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTWZ_A,SUROWIEC,0,80;CHARAKTER "CHARAKTER" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTZB_A,CHARAKTER,0,80;ROSLINNOSC "ROSLINNOSC" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTZB_A,ROSLINNOSC,0,80;ROSLIN_NIL "ROSLIN_NIL" true true false 80 Text 0 0,First,#,PL.PZGiK.337.0210__OT_PTZB_A,ROSLIN_NIL,0,80',
    add_source="NO_SOURCE_INFO"
)

In [ ]:
#konwersja warstw pokrycia terenu z poligonu na raster
arcpy.conversion.PolygonToRaster(
    in_features="kwzg_Merge",
    value_field="X_KOD",
    out_rasterdataset=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\kwzg_PolToRas",
    cell_assignment="CELL_CENTER",
    priority_field="NONE",
    cellsize=1,
    build_rat="BUILD"
)

In [ ]:
#reklasyfikacja rastra z warstwami pokrycia terenu do odpowiednich wartości kosztów jednostkowych
arcpy.ddd.Reclassify(
    in_raster="kwzg_PolToRas",
    reclass_field="X_KOD",
    remap="PTUT03 100;PTUT01 NODATA;PTTR02 1;PTTR01 20;PTZB01 200;PTZB02 100;PTZB05 50;PTZB03 200;PTZB04 200;PTWZ01 NODATA;PTWP02 200;PTWP03 NODATA;PTRK02 15;PTPL01 50;PTNZ01 150;PTNZ02 150;PTLZ01 100;PTLZ03 50;PTLZ02 50;PTKM01 100;NODATA NODATA",
    out_raster=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\Reclass_kw",
    missing_values="DATA"
)

In [ ]:
#przycięcie do obszaru gminy
Reclass_kw = arcpy.sa.ExtractByMask('Reclass_kw', gmina)
Reclass_kw.save(r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\Reclass_kw")

In [ ]:
#stworzenie mapy kosztów skumulowanych oraz mapy kierunków
out_distance_raster = arcpy.sa.CostDistance(
    in_source_data="gazociag",
    in_cost_raster=Reclass_kw,
    maximum_distance=None,
    out_backlink_raster=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\backlink",
    source_cost_multiplier=None,
    source_start_cost=None,
    source_resistance_rate=None,
    source_capacity=None,
    source_direction=""
)
out_distance_raster.save(r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\CostDis")

In [ ]:
#utworzenie ścieżki przyłącza gazociągu w postaci wektorowej dla wyniku o równych wagach
arcpy.sa.CostPathAsPolyline(
    in_destination_data="dz_wynik1",
    in_cost_distance_raster="CostDis",
    in_cost_backlink_raster="backlink",
    out_polyline_features=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\CostP_dzwynik1",
    path_type="EACH_ZONE",
    destination_field="FID",
    force_flow_direction_convention="INPUT_RANGE"
)

In [ ]:
#utworzenie ścieżki przyłącza gazociągu w postaci wektorowej dla wyniku o różnych wagach
arcpy.sa.CostPathAsPolyline(
    in_destination_data="dz_wynik2",
    in_cost_distance_raster="CostDis",
    in_cost_backlink_raster="backlink",
    out_polyline_features=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\CostP_dzwynik2",
    path_type="EACH_ZONE",
    destination_field="FID",
    force_flow_direction_convention="INPUT_RANGE"
)

In [ ]:
#przesortowanie wszystkich otrzymanych ścieżek od nakrótszej do najdłuższej(wynik z równymi wagami)
arcpy.management.Sort(
    in_dataset="CostP_dzwynik1",
    out_dataset=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\CostP_Sort1",
    sort_field="Shape_Length ASCENDING",
    spatial_sort_method="UR"
)
#wybranie najkrótszej ścieżki przyłącza i zapis do nowej warstwy shp
najkrotszwynik1 = arcpy.analysis.Select(
    in_features="CostP_Sort1",
    out_feature_class=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\najkrotszywynik1",
    where_clause="OBJECTID = 1"
)
arcpy.management.CopyFeatures(najkrotszwynik1, r"C:\Rok3\analizyprzestrzenne\1\najkrotszwynik1.shp")

In [ ]:
#przesortowanie wszystkich otrzymanych ścieżek od nakrótszej do najdłuższej(wynik z różnymi wagami)
arcpy.management.Sort(
    in_dataset="CostP_dzwynik2",
    out_dataset=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\CostP_Sort2",
    sort_field="Shape_Length ASCENDING",
    spatial_sort_method="UR"
)
#wybranie najkrótszej ścieżki przyłącza i zapis do nowej warstwy shp
najkrotszwynik2 = arcpy.analysis.Select(
    in_features="CostP_Sort2",
    out_feature_class=r"C:\Users\Iza\Documents\ArcGIS\Projects\MyProject3\MyProject3.gdb\najkrotszywynik2",
    where_clause="OBJECTID = 1"
)
arcpy.management.CopyFeatures(najkrotszwynik2, r"C:\Rok3\analizyprzestrzenne\1\najkrotszwynik2.shp")